In [1]:
import sys
import numpy as np
sys.path.append('C:/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')
from quantum_kernel.code.feature_maps import IQP
from quantum_kernel.code.feature_maps import Heisenberg
from quantum_kernel.code.utils import get_quantum_kernel,get_projected_quantum_kernel
from quantum_kernel.code.quantum_kernels import projected_kernels
from quantum_kernel.code.utils import get_dataset
from qiskit.circuit import Parameter,ParameterVector
from qiskit.circuit.library.data_preparation.pauli_feature_map import PauliFeatureMap
from sklearn.svm import SVC

In [2]:
test=IQP.Sparse_IQP(6,1,"full",1,int_time_scale=0.2)
#test=PauliFeatureMap(6,2)

In [3]:
test_kernel=get_projected_quantum_kernel(test,simulation_method='density_matrix',batch_size=5000)
test_kernel2=get_quantum_kernel(test,simulation_method='statevector')

In [4]:
x_train, x_test, y_train, y_test = get_dataset('plasticc', 6, 800, 200)

In [5]:
kernel_matrix=test_kernel.evaluate(x_vec=x_train,y_vec=x_train)

<class 'qiskit.result.result.Result'>


TypeError: _compute_overlap() missing 1 required positional argument: 'measurement_basis1'

In [ ]:
#qsvc = SVC(kernel='precomputed')
#qsvc.fit(kernel_matrix, y_train)
#score = qsvc.score(kernel_matrix, y_test)
#print(f"Score: {score}")

ValueError: Found input variables with inconsistent numbers of samples: [2, 800]

In [ ]:
#kernel_matrix2=test_kernel2.evaluate(x_vec=x_train,y_vec=x_train)

In [ ]:
#qsvc = SVC(kernel='precomputed')
#qsvc.fit(kernel_matrix2, y_train)
#score = qsvc.score(kernel_matrix2, y_test)
#print(f"Score: {score}")

In [ ]:
mus, nus = np.triu_indices(x_vec.shape[0], k=1)  # remove diagonal

NameError: name 'np' is not defined

In [ ]:
x_vec=x_train
y_vec=x_test
a=np.concatenate((x_vec, y_vec))

In [ ]:
x_vec.shape

(800, 6)